This notebook will load three different classifiers to label tweets hierarchically at 3 different levels. First, they will be sorted into behavior-related vs. not related. Second, they will be classified into 1st person vs. not 1st person. Third, a label will be assigned to first-person, behavior-related tweets that show time-related activity in the past, present, or future. 

First, we load the requisite modules into the environmment and change our base directory.

In [1]:
# ! pip install pbs
# import pbs
# print(pbs.__version__)
print()
# ! pip install pipelines

In [2]:
import os
import pickle
# import sklearn
# !pip install --upgrade pip
# !dir
# !pip install --upgrade sklearn
import sklearn
print(sklearn.__version__)
import pandas
import numpy
import sys
# import pipelines
# from sklearn.pipeline import FeatureUnion, Pipeline
# sys.modules['pipelines'] = Pipeline

0.21.1


In [3]:
# os.chdir('/Users/dethf/NYU/Spring 2019/Chunara Internship/nyu-twipsy/')
print(os.path.abspath(os.curdir))
os.chdir(os.path.abspath(os.curdir))
# os.chdir(sys.path[0])

C:\Users\eddie\Desktop\Rumi\alcohol


Next, we load the pickle files which contain our classifiers. 

In [4]:
# clf_alc    = pickle.load(open('/Users/dethf/NYU/Spring 2019/Chunara Internship/nyu-research/notebooks/python/classifiers/clf_alc_UPDATED.p', 'rb'))
# clf_fpa    = pickle.load(open('/Users/dethf/NYU/Spring 2019/Chunara Internship/nyu-research/notebooks/python/classifiers/clf_fpa_UPDATED.p', 'rb'))
# clf_fpl_dl = pickle.load(open('/Users/dethf/NYU/Spring 2019/Chunara Internship/nyu-research/notebooks/python/classifiers/clf_fpl_double_labeled', 'rb'))

# clf_fpa    = pickle.load(open('tobacco classifiers/clf_fpa_UPDATED.p', 'rb'))
clf_alc    = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
clf_fpa    = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
clf_fpl_dl = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))


C:\Users\eddie\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version pre-0.18 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\eddie\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator TfidfVectorizer from version pre-0.18 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\eddie\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator Pipeline from version pre-0.18 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\eddie\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator FeatureUnion from version pre-0.18 when using

In [5]:
# set the threshold: 99% of data is not alcohol related
#! jonathan comment: not really sure what this does. 
#! I know it's related to the 1% of tweets related to alcohol, but not much else. 
#clf_alc.steps[2][1].class_weight = {0:0.99, 1:0.01}
#clf_fpa.steps[2][1].class_weight = {0:0.5, 1:0.5}
#clf_fpl_dl.steps[2][1].class_weight = {0:0.99, 1:0.01}

Here we will use some pre-classified tweets and regenerate the alcohol-related vs. not alcohol-related, first-person vs. not first-person, and first-person-level labels. Since the data were used to train the classifier initially and were hand-annotated by MTurk workers, We should not expect to come up with identical labels for these levels of classification.

In [6]:
df = pandas.read_csv("./data/alcohol_training_instances.csv")

In [39]:
# printing the first 5 tweets to check content
df.text[0:5]

0    Impatiently waiting to get our hands on the ne...
1    Beer fans need their @ColumbusBrewing Bodhi. I...
2    Beer Olympics with my #1 fan and babe😍❤️ #Team...
3    Stone Cold use to be the baddest MF in my book...
4    Now @iamjohnoliver has to drink a Bud Light Li...
Name: text, dtype: object

In [42]:
%%time
## running the classifiers
# alcohol vs. not alcohol related
pred_alc= clf_alc.predict(df)
print(len(pred_alc))

# first-person vs. not first-person
#pred_alc_fp = clf_fpa.predict(df)

# first person level
#pred_fpl_dl = clf_fpl_dl.predict(df)

15651
Wall time: 4.2 s


We now check to see how well our classifier did reproducing the same labels as the pre-classified tweets.

In [52]:
alc_labels = []
# extracting pre-labeled classification
for i in range(0,len(df.labels)):
    alc_labels.append(df.labels[i][12])
# print(alc_labels[:10])

alc_labels = list(map(int, alc_labels))

# testing to see if the classifier generated the same labels    
comps = alc_labels != pred_alc
# print(len(comps))
comps = list(numpy.where(comps)[0])
print(comps)
print('pretrained alcohol classifer:  %d different item in whole dataset'%(len(comps)))

[2554, 2750, 7824, 9284, 9938, 11069, 12959, 13600]
pretrained alcohol classifer:  8 different item in whole dataset


Next we check to see what mismatched tweets there are and their content. 

In [53]:
# printing the mismatched labels and tweets
for i in comps:
    print('index %d'%(i))
    print(df.text[i], '\n', 'original label:', alc_labels[i], 'prediction:', pred_alc[i], '\n \n')

index 2554
Purple Wedding Drank
#GameOfThronesFood @midnight 
 original label: 1 prediction: 0 
 

index 2750
I'm a little drunk on you and high on summertime 
 original label: 1 prediction: 0 
 

index 7824
❤️😜
Drinking 
 original label: 0 prediction: 1 
 

index 9284
BEST #ATLANTA #REAL #ESTATE
#Chateau #Elan #Winery #Resort #GA 
2 Ac  ESTATE
info@cbmsinc.com 
http://t.co/Rl5gNnIeZl http://t.co/R1NwrUGTcO 
 original label: 1 prediction: 0 
 

index 9938
My #TeenChoice nominee for #ChoiceMaleWebStar is @TheMattEspinosa http://t.co/igxiscmsLY x1.017 
 original label: 1 prediction: 0 
 

index 11069
&lt;Creedy&gt; fucking jesus
&lt;Creedy&gt; i have to kill that bitch
&lt;logicalentity&gt; jesus?
&lt;logicalentity&gt; the romans beat ya 
 original label: 1 prediction: 0 
 

index 12959
It's better to love and then lost than to never have loved at all
If that was true I wouldn't be drinking this alcohol 🎶 
 original label: 0 prediction: 1 
 

index 13600
@NiallOfficial The show in Brusse

We see that there are 8 mismatched tweets. It seems remarkably accurate in reproducing the same labels as the human-annotated ones. 

We can check a different way to see if the classification was agreement with the original labels by subsetting the original labeled dataset by the predictions. We should see only tweets starting with the 'alcohol': 1 label, for the most part. 

In [60]:
# actually don't get it. it seems subset doesnt subsetting the original labeled dataset by the predictions
# subset = list(map(bool, pred_alc))
# print(len(subset))
# print(subset[:14])
# print(pred_alc[:14])
# df.iloc[subset,].labels[0:14]
# df.iloc[subset,][:14]

It looks like this initial classification was successful. 

Here we will subset our dataframe to only include alcohol-related tweets but also exclude tweets with having solely the "'alcohol': 1" label. We will then extract the first-person labels.

In [61]:
print(df.labels[:15])
subset_vec_alc_fp = []
for i in range(len(df)):
    if "'alcohol': 1" and "'first_person': 1" in df.labels[i]:
        subset_vec_alc_fp.append(1) 
    elif "'alcohol': 1" and "'first_person': 0" in df.labels[i]:
        subset_vec_alc_fp.append(1)
    else:
        subset_vec_alc_fp.append(0)

0                                        {'alcohol': 0}
1     {'alcohol': 1, 'first_person': 0, 'alcohol_rel...
2     {'alcohol': 1, 'first_person_level': 0, 'first...
3     {'alcohol': 1, 'first_person': 0, 'alcohol_rel...
4     {'alcohol': 1, 'first_person': 0, 'alcohol_rel...
5     {'alcohol': 1, 'first_person_level': 0, 'first...
6     {'alcohol': 1, 'first_person_level': 2, 'first...
7     {'alcohol': 1, 'first_person_level': 2, 'first...
8                                        {'alcohol': 0}
9     {'alcohol': 1, 'first_person': 0, 'alcohol_rel...
10                                       {'alcohol': 0}
11    {'alcohol': 1, 'first_person_level': 1, 'first...
12    {'alcohol': 1, 'first_person': 0, 'alcohol_rel...
13    {'alcohol': 1, 'first_person_level': 0, 'first...
14    {'alcohol': 1, 'first_person': 0, 'alcohol_rel...
Name: labels, dtype: object


In [68]:
# this shows how many tweets had the first-person classifier run on them.
sum(subset_vec_alc_fp)
# print(len(subset_vec_alc_fp))

9816

We will now extract the first-person labels from each of the original alcohol-related training tweets.

In [69]:
fp_lab = []

fp_0 = "'first_person': 0"
fp_1 = "'first_person': 1"

# the actual extraction
for i in range(len(df)):
    if fp_0 in df.labels[i] or fp_1 in df.labels[i]:
        if df.labels[i].find(fp_0) != -1:
            fp_lab.append(df.labels[i][df.labels[i].find(fp_0) + len(fp_0) - 1])
        else:
            fp_lab.append(df.labels[i][df.labels[i].find(fp_1) + len(fp_0) - 1])
print(fp_lab[:50])
# print(subset_vec_alc_fp[:10])
print(len(fp_lab))

['0', '1', '0', '0', '1', '1', '1', '0', '1', '0', '1', '0', '0', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1']
9816


Having extracted the original labels, we will now run the first-person classifier only on the tweets that were labeled originally. The classifier will not be run on tweets with solely the 'alcohol': 1 label. 

In [73]:
%%time
subset_vector = list(numpy.where(subset_vec_alc_fp)[0])
print(subset_vector[:10])
print(len(subset_vector)) # 9816

# print(df.iloc[subset_vector,][:10])
alc_fp_preds = clf_fpa.predict(df.iloc[subset_vector,])

[1, 2, 3, 4, 5, 6, 7, 9, 11, 12]
Wall time: 1min 29s


In [76]:
print(len(alc_fp_preds))
print(alc_fp_preds[:10])

9816
[1 1 1 1 1 1 1 1 1 1]


In [16]:
%%time
# array of probabilities
alc_fp_probs = clf_fpa.predict_proba(df[numpy.array(subset_vec_alc_fp) == 1])

In [74]:
print(len(alc_fp_probs))
print(alc_fp_probs[:10])

9816
[[0.4863973  0.5136027 ]
 [0.38392558 0.61607442]
 [0.47635895 0.52364105]
 [0.5        0.5       ]
 [0.16786706 0.83213294]
 [0.17335971 0.82664029]
 [0.14116919 0.85883081]
 [0.3576541  0.6423459 ]
 [0.11937854 0.88062146]
 [0.56474672 0.43525328]]


Here we will compare the original labels we extracted to the classifier's predictions. We will check to see what the rate of agreement is between our newly-generated predictions and the original annotations (labels).

In [17]:
fp_lab = list(map(int, fp_lab))
disag = alc_fp_preds != fp_lab
1-sum(disag)/len(disag)
#disag_subset = list(map(bool, disag))

#disag_idx = df.iloc[subset_vector,].iloc[disag_subset,].index.tolist()

#for i in disag_idx:
#    print(df.text[i], "\n original label:", fp_lab[i],"\n prediction:", alc_fp_preds[i], "\n\n")

0.7512224938875306

Our rate of agreement is about 75%. We now look at the tweet content as well as the original labels, the predictions, and the probabilities.

In [18]:
alc_fp_tweets = df[numpy.array(subset_vec_alc_fp) == 1].text
alc_fp_probs_round = numpy.around(alc_fp_probs,2).tolist()

for tweet, label, pred, prob in zip(alc_fp_tweets, fp_lab, alc_fp_preds, alc_fp_probs_round):
    print(tweet, "\n", "Original label: ", label, "Prediction: ", pred, "\n", "Probabilies: ", prob, "\n\n")

Beer fans need their @ColumbusBrewing Bodhi. I'm making another @tdg_powell trip for the #MemorialTournament http://t.co/Z9u3NrCBea 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.49, 0.51] 


Beer Olympics with my #1 fan and babe😍❤️ #TeamJamaica http://t.co/2hAT4vDQUT 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.38, 0.62] 


Stone Cold use to be the baddest MF in my book lol dude use to get hype; whip your ass and waste a case of beer in the ring 😂 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.48, 0.52] 


Now @iamjohnoliver has to drink a Bud Light Lime #SeppBlatter 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.5, 0.5] 


Drinking a Soft Parade by @ShortsBrewing @ Lake Johnson — http://t.co/XedrcSi9SS 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.17, 0.83] 


Its only Tuesday and I need a drink or four. 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.17, 0.83] 


jess told me no puking this friday.. so that means no getting drunk... 

 Original label:  1 Prediction:  1 
 Probabilies:  [0.21, 0.79] 


You will never hear me say i want a one/small glass of wine, just being honest. 🍷#ilovewine 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.45, 0.55] 


If my wife's hobbies don't include drinking beer and watching hockey while complaining about our friends who don't drink, it won't last 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.52, 0.48] 


Why why would Princess tag me in a drunk photo on Facebook I'm crying 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.4, 0.6] 


Drinking a Nut Brown Ale by @peakbrewing @ Berkeley Trace Apartments — http://t.co/QLMzDfCNPD 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.33, 0.67] 


Drinking a Belmont Abbey Robust Porter by @highlandbrews at @ncbeertemple — http://t.co/7HcUacuRdo 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.2, 0.8] 


Drinking a Citra Session IPA by @GreenFlashBeer at @starsrooftop — http://t.co/u5voxP40Yc 
 Original label

 Original label:  1 Prediction:  1 
 Probabilies:  [0.45, 0.55] 


Storm King and The Match Game my life rocks - Drinking a Storm King by @VictoryBeer @ Holiday Cafe  — http://t.co/9lHaJsqzMl 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.11, 0.89] 


I love beer, beer loves me. 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.23, 0.77] 


I know you've been drinking but I still need your ID. 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.3, 0.7] 


Drunk off sleepy time tea 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.2, 0.8] 


About to go to this lil bbq and have a couple drinks I guess 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.1, 0.9] 


Drinking a Resurrection by @brewersart at @percystreet — http://t.co/DmNkRTDniB 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.22, 0.78] 


Shout out to @BradWIlhelm515 for giving me back alley Mother Bears so I can drink longer tonight. YOU DA BES, AND YOU HAVE MY VOTE FOR MAYOR 
 Original label

 Original label:  0 Prediction:  1 
 Probabilies:  [0.49, 0.51] 


Just added Wild Beer Somerset Saison bottles. See our full beer menu: http://t.co/vugGtql77v #BeerMenus 
 Original label:  0 Prediction:  0 
 Probabilies:  [0.91, 0.09] 


Drinking a Bull Island by @wabashbrewing @ Wabash Brewing — http://t.co/7YLLfuEzD5 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.17, 0.83] 


Ice House Inn has a great selection of craft beer on tap and big glasses to drink it in. Check out the Beach @BeachShack330 out back! 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.43, 0.57] 


I really need to stop drinking ventis when i haven't eaten anything 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.15, 0.85] 


All I want to do is put a drink in my hand 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.09, 0.91] 


Whatever were drinking right now got me like ayyy 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.13, 0.87] 


Celebratory drinks!! @ Bakersfield Mass Ave

 Original label:  1 Prediction:  1 
 Probabilies:  [0.33, 0.67] 


Nina so drunk she can't open the door 😂😂 
 Original label:  0 Prediction:  0 
 Probabilies:  [0.74, 0.26] 


Drinking every night, cause we drink to my accomplishments 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.13, 0.87] 


quits: ReD_JaCk (john@mask-29052.intrstar.net) (Quit: Errection killed by beer) 
 Original label:  0 Prediction:  0 
 Probabilies:  [0.65, 0.35] 


Bittersweet kind of day, looking forward to drinking and streaming games with friends this weekend 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.16, 0.84] 


So much for not drinking this month 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.21, 0.79] 


hateeee waking up early after drinking 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.08, 0.92] 


Drinking a MEIYON LAGER by @Y_Market_Brew @ West Harlem Ski Area — http://t.co/Rfz9v5280F 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.27, 0.73] 


Like basicall

...

 Original label:  1 Prediction:  1 
 Probabilies:  [0.34, 0.66] 


Another update.  This was the best high alcohol IPA I have had yet.... (Hop-de-Ranged) http://t.co/qx8fKlpBrU 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.38, 0.62] 


@tran_christy no, who drinks at 1 on a Saturday you're so weird 
 Original label:  0 Prediction:  0 
 Probabilies:  [0.66, 0.34] 


Wine time (@ South River Winery &amp; Vineyard in Geneva, OH) https://t.co/3BNCaCbrt1 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.52, 0.48] 


If you kill me at work you win one free drink of your choosing. Any size. I'm there from 12:45-5:30 today. 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.6, 0.4] 


Drinking an Alpha King by @3floyds @ Hopson Hill Brewpub &amp; Lounge — http://t.co/qQyYpq8GuN #photo 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.2, 0.8] 


Tomorrow we celebrate THE DAD! Dads BYOB (bring your own baby) and get 1/2 off craft beer &amp; wine from… https://t.co/BlTOlnqV

Ahhhhhhhhhh why did drink so much and so much ... 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.22, 0.78] 


@DannyDeraney Get inside quick for the free beer. 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.48, 0.52] 


I need to find someone who can hold her liquor and show me a fun night 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.3, 0.7] 


It's getting to the point where I don't know if you actually want to talk to me or you're just not sober. 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.56, 0.44] 


@ @manwhale_corona for making me feel better&gt;&gt;&gt; 😂😂 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.17, 0.83] 


Never eat too much before going to a kickback if you expect to drink... Trust me it will back fire 😭 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.33, 0.67] 


@Samy_15 @doitbig___ the food was good lol the drinks 😁 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.23, 0.77] 


Jesus Christ when don't I want t

@niemanmusic was excellent tonight. @ dba256 Wine Bar Lounge https://t.co/XRZVglzb7N 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.35, 0.65] 


dad puts his alcohol mixed drinks in milk jugs &amp; me not realizing I just poured it in my cereal &amp; started eating it. fml 😕 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.41, 0.59] 


@lost_one25 lmaooo you've never seen me drunk off henny I'm a different person lmaoooooo 😭😭😭😭😭 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.29, 0.71] 


*Walks into babe's house*
"Where's your dad keep his beer? " 
 Original label:  0 Prediction:  0 
 Probabilies:  [0.77, 0.23] 


You've had a great evening,haven't finished off the wine bottle Pour into an ice cube tray &amp; freeze Perfect for sauces, waste not want not! 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.35, 0.65] 


@TayRe you just assumed that at 1a.m that I'm automatically drunk... You stand corrected good sir!! 
 Original label:  1 Prediction:  1 
 Probab

Swing! So much fun last night at Jazz At Lincoln Center's Swing Summer Cocktail Party.… https://t.co/G55Cps7fOq 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.57, 0.43] 


I was hoping to be a victim of a qualifying crime this morning but the little drunk guy with his shopping cart kept his distance 😐 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.45, 0.55] 


@DuaneBS u better swallow that beer faster than you swallowed @Jordan_Palmer_ semen last weekend 😂 
 Original label:  0 Prediction:  0 
 Probabilies:  [0.66, 0.34] 


@ConnellCaleb She's being a little ho ho, just wait. I hate nick so much hahaha 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.5, 0.5] 


Make it a special day w/ @BesseratB #champagne @WineSpectator http://t.co/cYh0TGnr1w 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.56, 0.44] 


Playing dominoes and drinking...#lifeisgood along with 200 followers....thanks to @MattieLBreaux http://t.co/NhYDO88m4g 
 Original label:  1 Prediction:  1


@quixoticideal he's going to catch feelings like....well...like anyone who listens to Sam Smith after a night of 3rd wheeling and drinking. 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.23, 0.77] 


went from fallin in love to drunk and fallin apart 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.31, 0.69] 


Drink for a cause!  We are raising funds for surfrider, protecting our coast and environment. We are… https://t.co/GZYrmr9hE3 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.5, 0.5] 


Alcohol makes everything so much.... Better? 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.41, 0.59] 


drank up in my cup but that don't mean that I fuck slowww 
 Original label:  0 Prediction:  1 
 Probabilies:  [0.16, 0.84] 


@Mo_was_there_yo @BlackPplVines @iplay_keys96 @Hi_Sadiess @tashnnv STG I feel like I'm all in one when I'm drunk and sober 😂 
 Original label:  1 Prediction:  1 
 Probabilies:  [0.25, 0.75] 


I got the love birds chirpin at the window but I

On first glance it seems that when the classifier disagrees with the original label, the probabilities of either 0 or 1 are quite close. We will look at that step next.

In [19]:
alc_fp_disag_tweets = df[numpy.array(subset_vec_alc_fp) == 1][numpy.array(alc_fp_preds) != numpy.array(fp_lab)].text
alc_fp_disag_labels = numpy.array(fp_lab)[numpy.array(alc_fp_preds) != numpy.array(fp_lab)]
alc_fp_disag_preds  = numpy.array(alc_fp_preds)[numpy.array(alc_fp_preds) != numpy.array(fp_lab)]
alc_fp_disag_probs  = numpy.array(alc_fp_probs_round)[numpy.array(alc_fp_preds) != numpy.array(fp_lab)]

for disag_tweet, disag_label, disag_pred, disag_prob in zip(alc_fp_disag_tweets, alc_fp_disag_labels, alc_fp_disag_preds, alc_fp_disag_probs):
    print(disag_tweet, "\n", "Original label: ", disag_label, "Prediction: ", disag_pred, "\n", "Probabilities: ", disag_prob, "\n\n")

Beer fans need their @ColumbusBrewing Bodhi. I'm making another @tdg_powell trip for the #MemorialTournament http://t.co/Z9u3NrCBea 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.49 0.51] 


Stone Cold use to be the baddest MF in my book lol dude use to get hype; whip your ass and waste a case of beer in the ring 😂 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.48 0.52] 


Now @iamjohnoliver has to drink a Bud Light Lime #SeppBlatter 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.5 0.5] 


Sweeter than muska dime wine 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.36 0.64] 


Business hours are between 1 1/2 and 5 drinks. ;) 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.56 0.44] 


So for now, no drinking for the foreseeable future! 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.38 0.62] 


Drinking an Osiris Pale Ale by @SunKingBrewing @ Union Station — http://t.co/7eDLi4vkF0 
 Original label:  0 Prediction:  1 
 Probabiliti

 Original label:  0 Prediction:  1 
 Probabilities:  [0.61 0.39] 


I forgot how much I fucking hate beer 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.13 0.87] 


I picked up my moms drunk ass from blue 32 and took her to the gas station to get me drinks and she stopped to talk to lil kids outside 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.54 0.46] 


Well since I'm dd I can't get drunk but I will get high af 💃🏽 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.07 0.93] 


I always get sleepy when I'm on that's why I don't drink😂 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.28 0.72] 


Ive been drinking too much benadryl to go to sleep..i need it though i think too much when its time to sleep :( 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.11 0.89] 


I like my boyfriend when his drunk lmao 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.27 0.73] 


When Gavin makes me his drinking partner without my knowledge💀 
 Origina

 Original label:  0 Prediction:  1 
 Probabilities:  [0.31 0.69] 


Drinking a Hoosier Ale by @hoosierbrewco at @tomtaproom — http://t.co/ht8V94lmXP #photo 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.19 0.81] 


Let's drink and do stupid stuff 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.34 0.66] 


Drinking an IPA by @stbcbeer @ Blue Ridge Trail Golf Course — http://t.co/5bPp0V62Jf #photo 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.23 0.77] 


Talking wedding at @thorndalecraft - Drinking an Union Jack IPA by @FirestoneWalker @ The Craft House  — http://t.co/9rX6oQqDDk 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.22 0.78] 


When the tattoo artist buys you a round of drinks 
 Original label:  1 Prediction:  0 
 Probabilities:  [0.64 0.36] 


Drinking beer is my thing. But I don't believe in smoking. Give me some damn whiskey or cigarettes! I don't care! I smoke some but w/e! 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.37 0

it's ok to be drunk on the lake @ High Rock Lake, NC https://t.co/h4tP2B6v6m 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.53 0.47] 


Bought a Adam Lallana jersey. Cant decide if I should buy a mini fridge for my room to fill with beer or a puppy. 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.32 0.68] 


Drinking an Old 21 by @brewkettle at @localroots09 — http://t.co/f1DTZcqA5E 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.37 0.63] 


"I'm not drunk I just did twenty jumping jacks" 😂😂 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.14 0.86] 


A drunk guy just told me I have the cutest nostrils he's ever seen and I have the perfect nose for a nose ring ONG AW #blessed 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.48 0.52] 


Drinking a First Anniversary IPA by @OtherHalfNYC at @mugsalehouse — http://t.co/SBKRgHr7qE 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.28 0.72] 


its too early for my family to be drunk already 



drinking on an empty stomach lmaoooo 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.21 0.79] 


I picture myself also having some sports related thought and the thought of having a beer before I realize what's going on. LOL 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.26 0.74] 


I have a fucking alcohol class for 4 hrs today 😭 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.32 0.68] 


There might be a shortage of Corona beer this summer.  And even worse news:  there will be a surplus of Natty Light. 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.6 0.4] 


The chance of me wearing the same thing from yesterday totally relies on how much I drank last night 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.12 0.88] 


#supportyourlocaldive, 1696 2nd Ave btwn 87 &amp; 88. Drink specials all night! 8pm. 13 lines getting ready a beer explosion coming soon! 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.49 0.51] 


Drunk ugly chick 

...

 Original label:  0 Prediction:  1 
 Probabilities:  [0.52 0.48] 


@disneysquotess ...Drunk Roommates Are Bad Biz -Hahaa 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.53 0.47] 


@jaknowlden, pacify yourself with a drink. 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.56 0.44] 


Slow as the Mississippi, strong as a fifth of whiskey 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.58 0.42] 


I'll take all my food and/or drinks to go . Thank you 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.32 0.68] 


Definitely a sour beer but not overly sour which I like. I would... (Technicolor Motorhome) http://t.co/3XrhLFeXAl #photo 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.24 0.76] 


@RayalityCheck We're #notalwaysthirsty. But when we drink, "we like that Do-mestic beer." 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.48 0.52] 


Champagne and pasta and jazz OH MY #cheektocheek @ladygaga @ Joanne Trattoria https://t.co/Vtfu43V2N5

 Probabilities:  [0.54 0.46] 


Clear liquor makes me wanna throw up just looking at it. 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.33 0.67] 


When @king_manzo  drinks too much  https://t.co/qvTm6aDu0Q 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.5 0.5] 


Wine night @ Postino's http://t.co/XHwFufCfPO 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.34 0.66] 


Brunch: the socially acceptable excuse to drink before noon ☀️🍹 @ Sunny Spot https://t.co/BxRTX6ESsS 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.49 0.51] 


Jekyll n Hyde... This team makes me drink! https://t.co/8kg4ayMgNp 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.51 0.49] 


@parrillasassyaf LOL I hope not if Lana were drunk, but I'm so happy that she looks so happy be there. 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.55 0.45] 


#🍺 #beer @ The Boardwalk @ Brighton Beach https://t.co/dY2tEQcmCF 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.

 Probabilities:  [0.39 0.61] 


He finna be stupid drunk  https://t.co/WkG1zIma3s 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.46 0.54] 


*and spilling our drinks everywhere while we laugh at Kim's drinking habits (a lot of beer)  https://t.co/teyJkhnnLd 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.51 0.49] 


And he didn't even drink much cause I know he can hang.... then that morning we found he shit on himself all over the walls lol 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.55 0.45] 


Ciroc even though i really dfw drinking  https://t.co/De1mdZv0qR 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.36 0.64] 


@paigieebabiee or we can dress up and get drunk???? 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.41 0.59] 


@n_settle_ no drinking and driving is bad 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.48 0.52] 


@papabreadstick foremost liquor near South Pasadena. 
 Original label:  0 Prediction:  1 
 Probabili

 Probabilities:  [0.48 0.52] 


Boom. (@ World of Beer in Jacksonville, FL w/ @sassyfreshjazz) https://t.co/zojvI6vSnx 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.52 0.48] 


@LargeLocale When you get eaten by I T-Rex I'll pour some vodka out for you as I'm running away. 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.43 0.57] 


@angeljordan_yo if we get drunk, we be thinking we look like this.  https://t.co/iSXg35l8SV 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.3 0.7] 


@amyschumer OMG I'm #InsideAmy!  I'm A Little Short On 💸 Can I Borrow Some Vodka?Preferably Not The Vodka You Poured On Your Feet Though! 😂 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.49 0.51] 


@igobyhardy no but when I'm drunk with friends it's good ok😂😂💀 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.17 0.83] 


Coy: did you drink that moonshine? Huntyr: ........." Coy:" that's a question, we answer those." 😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂 
 Original label:  0 Predicti

 Probabilities:  [0.61 0.39] 


It's #WineWednesday at #CucinaRustica in #Sedona, AZ. Come enjoy a glass of wine and indulge in some fine Italian. http://t.co/IEZFnAuerG 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.53 0.47] 


I want to leave this doctors office, it's cold, I'm hungry &amp; sober. 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.3 0.7] 


@rijgod Sometimes he freestyles tho😭😭 , Bruh gets hella High and drinks hella lean and just gets In the booth fr. 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.37 0.63] 


I want to live in a world where alcohol is zero calories 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.58 0.42] 


Me when I get home drunk :-} jk I don't drink.. https://t.co/fd1XCeNXEr 
 Original label:  0 Prediction:  1 
 Probabilities:  [0.38 0.62] 


@FrozenSoda8 @Allentownmike @shaunholt8587 @ThinkingLWNJ @carlsbadbugkil1 fuck the beergarita -if i wanted a beer w lime, I'd order that 
 Original label:  0 Prediction:  1

It does seem like the classifier is never really sure of the prediction when it disagrees with the original label.

We next create a confusion matrix to get a better sense of where the classifier makes mistakes.

In [20]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(fp_lab, alc_fp_preds, labels=[0,1])
print(conf_mat)

[[1234 2225]
 [ 217 6140]]


It looks like only 217 tweets were mislabeled as not first-person when they were first-person. Compared to the 6140 that were labelled correctly, however, this does not seem like such a big error. What does seem significant is that fewer than half of the tweets originally labeled as not first-person were correctly predicted as such, and 2225 tweets (about 65% of the original not first-person tweets) were incorrectly predicted to be first-person. This might hint at a problem with this level of classifier that we might take a look at later. 

For completeness, we will take a look at the confusion matrix for the alcohol-related classifier that was remarkably accurate above:

In [21]:
conf_mat_alc = confusion_matrix(pred_alc, alc_labels)
print(conf_mat_alc)

[[ 5353     5]
 [    3 10290]]


We can see that this classifier performed about-evenly well classifying alcohol-related vs. not alcohol-related tweets. There were only 8 errors, as we saw before: 3 tweets were misclassified as not alcohol-related and 5 tweets were misclassified as alcohol-related.

We will finally look to see how well the classifier reproduces the first-person-level labels from the original training dataset. 

In [22]:
# appending labels based on whether tweet label contains phrase
fpl_lab = []
fpl_subset_vec = []

fpl = "'first_person_level'"

# extracting FP levels and constructing subset vector simultaneously
for i in range(len(df)):
    if fpl in df.labels[i]:
        fpl_lab.append(int(df.labels[i][df.labels[i].find(fpl) + len(fpl) + 2]))
        fpl_subset_vec.append(1)
    else:
        fpl_subset_vec.append(0)

In [23]:
# running the classifier on the subset we defined above
fpl_idx_vec = list(numpy.where(fpl_subset_vec)[0])
alc_fpl_preds = clf_fpl_dl.predict(df.iloc[fpl_idx_vec,])

In [24]:
# predicting probabilities for FPL for use later
alc_fpl_probs = clf_fpl_dl.predict_proba(df.iloc[fpl_idx_vec,])

In [25]:
fpl_disag = list(map(int, fpl_lab != alc_fpl_preds))
# printing rate of disagreement
sum(fpl_disag)/len(fpl_disag)

0.3312883435582822

We have about 33% disagreement rate. This might seem poor but considering the fact that the classifier only labels up to 3 levels (0, 1, and 2), while the annotated labels had 4 levels (0, 1, 2, 3), this is not so bad. The classifier may not be working properly, however. We will check that now. 

In [26]:
fpl_tweets_disag = df.iloc[fpl_idx_vec,].iloc[list(numpy.where(fpl_disag)[0]),].text
fpl_disag_vec = list(numpy.where(fpl_disag)[0])

for i in fpl_disag_vec:
    print(df.iloc[fpl_idx_vec,].iloc[i,].text, '\n', "Original label: ", fpl_lab[i], "Prediction: ", alc_fpl_preds[i], '\n')

Its only Tuesday and I need a drink or four. 
 Original label:  2 Prediction:  1 

jess told me no puking this friday.. so that means no getting drunk... ouch 😩 
 Original label:  2 Prediction:  0 

I'll be fine just drinking my wine 
 Original label:  2 Prediction:  0 

Wtf was I drinking Sunday?😂🔫🙊 http://t.co/3c2mz1Uz8L 
 Original label:  2 Prediction:  0 

Drinking a Café Deth by @revbrewchicago at @rootspizza — http://t.co/ni9qZl4N9S 
 Original label:  2 Prediction:  0 

In a matter of 10 minutes, whilst trying to be productive, I managed to shatter a wine glass and a bottle of rosé. 😫😫😫 
 Original label:  0 Prediction:  1 

#NoteToSelf Don't drink on a Monday..😣 
 Original label:  2 Prediction:  0 

Whatever Wagner drank, ate, injected before the game needs to be administered again tomorrow. 
 Original label:  0 Prediction:  1 

next stop: sun, sand and a drink in my hand😉🍹 #poet 
 Original label:  0 Prediction:  1 

As much as I'd love to get black out drunk this weekend, I thin

Drinking an 33 by Bubba's 33 @ Bubba's 33 — http://t.co/57X9ZsvuxE 
 Original label:  2 Prediction:  0 

I Rather Be Your N I G G A So We Can Drunk And Smoke Weed All Day. 
 Original label:  2 Prediction:  0 

Females don't get shit sometimes man 😴 
 Original label:  1 Prediction:  2 

Who's tryna drink? 
 Original label:  2 Prediction:  1 

I'm hungover half the time and drunk the  other half 
 Original label:  2 Prediction:  0 

I'm a little drunk on you and height on summertime 🎶 
 Original label:  2 Prediction:  0 

He just drunk a 6oz 🍼 now he sleep!! 👶😴😴 
 Original label:  2 Prediction:  0 

I drink liquor and eat ice cream...Sooooo whatttt 
 Original label:  2 Prediction:  1 

i said i wasn't drinking tonight 
 Original label:  0 Prediction:  1 

I need a public chicken tender sub rn 
 Original label:  0 Prediction:  1 

The only thing to do around here is get drunk and mow your yard 
 Original label:  2 Prediction:  1 

let's eat these noodles, drink these beers and dance our h

 Original label:  0 Prediction:  1 

A little beer and pizza before @Metallica #xgames @XGames http://t.co/MHn5jXooWD 
 Original label:  1 Prediction:  0 

Just got hit on by a drunk middle aged wo(man) at a 21+ club while having dinner with my mom... What is life😂😂 http://t.co/wuvVhIKyCF 
 Original label:  2 Prediction:  0 

I didn't know my night was headed for Coach Keck buying me a bucket of beer &amp; busting out legendary dance moves. Now that's #winning 
 Original label:  0 Prediction:  2 

Omg @AmoHeals just yelled at me "OMG I want a sloppy bottom!"
Laughed so hard I had to go to the floor.

(It's a drink name FYI) 
 Original label:  2 Prediction:  1 

When you're drunk and really want red lobster so you try to look up their hours.... http://t.co/GcERVN4s6S 
 Original label:  2 Prediction:  0 

I just paid $9 for a beer... I hate bars. 
 Original label:  0 Prediction:  1 

Only thing I hate about drinking is having to use the restroom every 15 minutes 😁 
 Original label:  0 Pr

I miss my @LeaBlackMiami I need her #turbans in my life.  When you are in Houston please meet me for a drink #needuinmylife #selfmadebitch 
 Original label:  1 Prediction:  2 

@EvanDonovanNews your welcome love meet up some time chat have a beer local guy grew up mars hill still live mars hill . 
 Original label:  2 Prediction:  1 

Drunk on the beach takes me back 😭😭 
 Original label:  2 Prediction:  0 

Doing research and trying to plan my life out turned into getting drunk with my mom real fast 
 Original label:  2 Prediction:  0 

I drank some wine and now I have pre risky text syndrome. 
 Original label:  0 Prediction:  1 

All I wanna do right now is have peaches and wine in Italy 
 Original label:  2 Prediction:  1 

If it's Molsen Canadian, drink one for me 
 Original label:  2 Prediction:  1 

I be sending drunk texts and be sober af 😂 
 Original label:  0 Prediction:  1 

7. BUT MY DRUNK ASS WAS AGGRESSIVELY AND REPEATEDLY TEXTING HIM BC I WANTED ATTENTION. So he finally tel

Dude stop starin me down...you're drinking Smirnoff ice. 
 Original label:  1 Prediction:  2 

Pours a drink alone w my sick ass garlic and onion meal from out of nowhere, raises it, a toast! to me!!! B) 
 Original label:  0 Prediction:  2 

When you have your first drink all week. Shitz like crack 
 Original label:  1 Prediction:  2 

OITNB with pizza and beer and @Gingerhazing 🙌🙌🙌 
 Original label:  1 Prediction:  0 

Ladies coming through for beer pong 😁 it's lit! 🔥🔥🔥 
 Original label:  0 Prediction:  1 

I pop champagne with who I'm closest too. 
 Original label:  2 Prediction:  0 

I gotta say, it's a bit rough going out &amp; getting drunk, seeing so many cute girls at the bar while being married. Quick fantasies abound! 😍 
 Original label:  2 Prediction:  0 

I just saw my next door neighbor who's a city cop working downtown and I drunkenly spoke more to him tonight then I ever have sober 
 Original label:  2 Prediction:  1 

Drink until you forget what made you drink in the fir

...

 Original label:  1 Prediction:  2 

@WhiteDoggMike @ManfredandSons I hate dark beer :( 
 Original label:  2 Prediction:  1 

@adame_ just went for drinks and it was great. Heard only good things about brunch 
 Original label:  0 Prediction:  2 

Can't get enough of Napa Valley @ Domaine Carneros Champagne Estate https://t.co/RUNMgi0XyZ 
 Original label:  2 Prediction:  1 

@brianbatten1 @iamshawnrice I agree! Nothing a little beer can’t solve! 
 Original label:  2 Prediction:  1 

Sober up with some tapas &amp; cava. (@ Bom Bolla in Chicago, IL) https://t.co/PCZ61uuBCE 
 Original label:  1 Prediction:  0 

Come cure that @phillybeerweek hangover with me @FieldHousePHL with some @VictoryBeer and @FederalDonuts 2pm! Movies, donuts, couches &amp; beer 
 Original label:  2 Prediction:  1 

Thank God for Klonopin and Gin. #ihateflying #6hourflight #nowboarding 
 Original label:  2 Prediction:  1 

Drinking coffee but wish I was pool side chillin with a mojito 
 Original label:  1 Predictio

 Original label:  0 Prediction:  1 

@Dusanek113 every time I go in there now it literally takes 20mins to get my drink even when it's not busy 😭 
 Original label:  2 Prediction:  1 

..It's that day &amp; hour of the week when "Wings'n Beers" taste oh so good! #HappyFriday 🍻😜 https://t.co/UHmjPR2hz8 
 Original label:  1 Prediction:  0 

We prolly shouldn't be texting, im always forgetin that lesson.I've had one too many to drink and all I can't think is to send u a message 😁 
 Original label:  2 Prediction:  1 

No matter what the situation is, I'm never letting another man pour champagne or any other liquid on me. https://t.co/PIo0kjg1VW 
 Original label:  2 Prediction:  0 

@Justinschillig beer before liquor, ur still getting drunk so don't be a pussy 
 Original label:  0 Prediction:  2 

@ChawlinLately @ketteringpolice he is planing a very large alcohol party tonight with citizens under 21. There also will be meth there. 
 Original label:  2 Prediction:  1 

i want to be a sober si

Jesus Christ when don't I want to get drunk, I'm going to become an alcoholic 
 Original label:  2 Prediction:  1 

*7 am*
"if you &amp; haley are ever drunk somewhere at 2 am and get lost driving, know: odd highways are north &amp; south, &amp; evens are east &amp; west 
 Original label:  1 Prediction:  0 

I'm taking my dad to the game. I told him it's about the beer and quality time with me. 💕 https://t.co/SiFdNIZTEL 
 Original label:  0 Prediction:  1 

Had my annual #rattlesnake roundup party yesterday. The day started with a #bloodymary bar featuring @Titos Vodka http://t.co/OLi84feVC7 
 Original label:  2 Prediction:  0 

time for a drink? #awnaw @ChrisYoungMusic 
 Original label:  2 Prediction:  1 

New ridgewoodwhiskeyassoc!! A group of like minded whiskey enthusiasts that gather in #ridgewood,… https://t.co/Za8JiJUIJq 
 Original label:  1 Prediction:  0 

@Foodiechats @TwoBrothersBeer @LUCKdallas has an AMAZING #coconut beer and this comes from a girl who really isn't a beer 

 Original label:  2 Prediction:  0 

Birthday week has commenced! Flower cred: akcheck (aka best boyfriend ever) Wine cred: myself (of… https://t.co/FxI0WkbCov 
 Original label:  1 Prediction:  0 

.@spyou73 I swear Joseph you are gonna drive me to drink, if you re-gram ONE more time! lol I wanna see ORIGINAL content boo! 
 Original label:  2 Prediction:  1 

@_SaskiaMitchell
Yup &amp; warm!
Love it.
Though I don't drink much.
Only on vacation &amp; s/times @ dinner
Nice after our Waterfront freeze, ay? 
 Original label:  0 Prediction:  2 

Greasy liquor party at my pad who is down 
 Original label:  1 Prediction:  0 

@DaveDTC @winetrain @TravelingWithS we just put some Czech wine on our list! I must say perfect light wines for this weather in NY. 
 Original label:  1 Prediction:  0 

@JesseLopez2150 @romero360luis yes because I smoke and drink lmao 
 Original label:  2 Prediction:  1 

#Champagne &amp; #Cigar Air Jordan Retro 7's 'Championship Pack' coming out tomorrow morning!!… htt

 Original label:  2 Prediction:  0 

@Joycommaashia lol we've never played beer pong? 👀😂😂😂 
 Original label:  2 Prediction:  0 

Hanging at #Cindy for a few drinks afterwork. Nice to have this just two blocks from @BlueMetaIInc… https://t.co/2TkeDyEOAg 
 Original label:  1 Prediction:  0 

Laguardia &amp; American Eagle Airlines are both my absolute least favorite right now. @karmagypsy can I come over for a drink plz? 
 Original label:  0 Prediction:  1 

So sad I can't drink this anymore 😭 http://t.co/3pQg9p0P8U 
 Original label:  2 Prediction:  0 

Super drinkable. Light and just sour enough with some citrus acidity. ... (Berliner Weisse) http://t.co/fHwl6avN7f #photo 
 Original label:  2 Prediction:  0 

#tocamadera Getting ready for this Sunday! We are going all out with the Food &amp; Drinks! #all_shots… https://t.co/Xky7JO63lZ 
 Original label:  1 Prediction:  0 

"It's 9 o'clock on a Saturday. Regular crowd shuffles in. There's an old man sitting next to me, making love to his 

We will now figure out which original labels correspond to the Current, Looking, and Reflecting labels from the paper.

In [27]:
numpy.bincount(fpl_lab)

array([3211, 1676, 1394,   76], dtype=int64)

The "1" and "2" labels correspond to the Looking and Reflecting labels, respectively. However, the Current label seems to have been split between labels "0" and "3." Why this is or whether this split occurred before or after the paper was published we cannot be sure, but the bin counts for these labels (3211 and 76) do add up to what was reported in the paper (3287). 

Next we try to isolate only the tweets that received a label of 3 to see what this label could mean based on the tweet content...

In [28]:
fpl_tweet_3 = df.iloc[fpl_idx_vec,].text[numpy.array(fpl_lab) == 3] # tweets that received "3" label
preds_3 =   alc_fpl_preds[numpy.array(fpl_lab) == 3] # predictions for which tweet received "3" label
fpl_probs_lab3 = numpy.around(alc_fpl_probs[numpy.array(fpl_lab) == 3],2).tolist() # list of probabilities

# printing tweet, original label, prediction, and probabilities
for tweet, pred3, prob in zip(fpl_tweet_3, preds_3, fpl_probs_lab3):
    print(tweet, "\n", "Original first person level: 3  Prediction: ", pred3, "\n", "Probabilities: ", prob, "\n\n")

he straight up drank from my bottle oh my god 
 Original first person level: 3  Prediction:  0 
 Probabilities:  [0.68, 0.06, 0.27] 


Why am I drinking a big cup of straight well gin rn 
 Original first person level: 3  Prediction:  0 
 Probabilities:  [0.98, 0.01, 0.01] 


We he a little horsey named Paul Revere just me and my homies and a quart of beer. 
 Original first person level: 3  Prediction:  0 
 Probabilities:  [0.97, 0.02, 0.01] 


Oh man, my head hurts so bad, I should have saved one beer for the hangover...🍺🔇 
 Original first person level: 3  Prediction:  2 
 Probabilities:  [0.15, 0.14, 0.71] 


ive had more drunk texts and than sober texts today thats cool too 
 Original first person level: 3  Prediction:  2 
 Probabilities:  [0.16, 0.01, 0.83] 


5 am drunk off da whiskey with these two rockstars @officialkukharrell @djanser !!! These guys always... https://t.co/w5In6pjvoU 
 Original first person level: 3  Prediction:  0 
 Probabilities:  [0.99, 0.01, 0.01] 


Sometime

Most of these tweets seemed to receive a "0" label as if the classifier is trying to correctly classify these "3"-labeled tweets, which is good. Looking at accompanying probabilities there does not seem to be a discernible pattern: prediction probability does not seem to be dependent on the label. Finally, it does not seem outwardly apparent what this "3" label would have been for based on the tweet text...

We will finally produce a confusion matrix telling us where exactly the classifier made mistakes.

In [29]:
conf_mat_alc_fpl = confusion_matrix(fpl_lab, alc_fpl_preds)
print(conf_mat_alc_fpl)
print(numpy.bincount(fpl_lab), numpy.bincount(alc_fpl_preds))

[[2538  396  277    0]
 [ 461 1086  129    0]
 [ 517  250  627    0]
 [  44   13   19    0]]
[3211 1676 1394   76] [3560 1745 1052]


In [30]:
print(round(2538/numpy.bincount(alc_fpl_preds)[0],2), 
      round(1086/numpy.bincount(alc_fpl_preds)[1],2),
      round(627/numpy.bincount(alc_fpl_preds)[2],2))

0.71 0.62 0.6


The classifier gets the Current label right about 71% of the time, the Looking label about 62% of the time, and the Reflecting label right about 60% of the time. 

We will try checking some of the probabilities for these tweets to see if they were deemed especially difficult by the classifier to predict a label for. 

In [31]:
alc_fpl_probs = clf_fpl_dl.predict_proba(df.iloc[fpl_idx_vec,])
fpl_probs_lab3 = numpy.around(alc_fpl_probs[numpy.array(fpl_lab) == 3],2).tolist()


In [32]:
fpl_probs_lab3 = numpy.around(alc_fpl_probs[numpy.array(fpl_lab) == 3],2).tolist()


-------------------------------------------------------
-----------------------

Yijun:

In [81]:
df.head()

,_id,created_at,labels,predict,text,user,user.created_at,user.favourites_count,user.followers_count,user.friends_count,user.statuses_count,user.verified
0,556e0ee3d6dfbb462880f0a5,Tue Jun 02 20:16:08 +0000 2015,{'alcohol': 0},0.526050,Impatiently waiting to get our hands on the ne...,{'created_at': 'Thu Jun 12 22:14:05 +0000 2014...,Thu Jun 12 22:14:05 +0000 2014,394,407.0,1997.0,823.0,False
1,556e128ad6dfbb46288111e4,Tue Jun 02 20:31:44 +0000 2015,"{'alcohol': 1, 'first_person': 0, 'alcohol_rel...",0.516649,Beer fans need their @ColumbusBrewing Bodhi. I...,{'created_at': 'Mon Oct 06 21:00:38 +0000 2008...,Mon Oct 06 21:00:38 +0000 2008,806,1006.0,960.0,10442.0,False
2,556e12e6d6dfbb462881153e,Tue Jun 02 20:33:15 +0000 2015,"{'alcohol': 1, 'first_person_level': 0, 'first...",0.523513,Beer Olympics with my #1 fan and babe😍❤️ #Team...,{'created_at': 'Tue Dec 20 02:46:19 +0000 2011...,Tue Dec 20 02:46:19 +0000 2011,11662,563.0,356.0,13940.0,False
3,556e1464d6dfbb4628812330,Tue Jun 02 20:39:37 +0000 2015,"{'alcohol': 1, 'first_person': 0, 'alcohol_rel...",0.502633,Stone Cold use to be the baddest MF in my book...,{'created_at': 'Sun Mar 11 08:22:56 +0000 2012...,Sun Mar 11 08:22:56 +0000 2012,860,703.0,684.0,89573.0,False
4,556e15f1d6dfbb4628813236,Tue Jun 02 20:46:14 +0000 2015,"{'alcohol': 1, 'first_person': 0, 'alcohol_rel...",0.535758,Now @iamjohnoliver has to drink a Bud Light Li...,{'created_at': 'Thu Jan 14 03:03:33 +0000 2010...,Thu Jan 14 03:03:33 +0000 2010,3473,9414.0,1486.0,16435.0,True


In [98]:
# split data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, df.labels, test_size=0.1, random_state=26)

In [99]:
# print(X_train[:5])
# X_train[:5]
# X_test[:5]
y_train[:5]
y_test[:5]
print('training set:', len(X_train))
print('testing set:', len(X_test))


training set: 14085
testing set: 1566


------------------------
This section is for code that may be used later...

In [33]:
#preds = clf_alc

#preds = clf_fpt.predict(pd_test)
#print(preds)

#for tweet, prediction in zip(tweets, preds):
#    print(tweet, '\n',prediction, '\n \n')

from operator import eq

list1 = [1,2,3] 
list2 = [4,5,6]

list(map(eq, list1, list2))

[False, False, False]

In [34]:
x = clf_alc.named_steps["clf"]
y = clf_alc.named_steps['features']

In [35]:
x.coef_

array([[-2.53405597,  0.87940841,  5.56477366, ..., -3.24180786,
        -1.35429585,  2.02874986]])

In [36]:
# trying to extract labels here
fpl_lab = []

fpl = "'first_person_level'"

# extracting FP levels
for i in range(len(df)):
    if fpl in df.labels[i]:
        fpl_lab.append(int(df.labels[i][df.labels[i].find(fpl) + len(fpl) + 2]))
    else:
        fpl_lab.append(-1) # adding -1 here to mark where no label was given
        
# subsetting to run the classifier only on tweets that received a FPL label originally
fpl_subset_vec = []

for i in range(len(fpl_lab)):
    if fpl_lab[i] != -1:
        fpl_subset_vec.append(1)
    else:
        fpl_subset_vec.append(0)

In [37]:
# getting index for disagreement tweets (FPL)
fpl_idx_disag = df.iloc[fpl_idx_vec,].iloc[list(numpy.where(fpl_disag)[0]),].index.tolist()

In [38]:
#print(fp_lab[0:10], alc_fp_preds[0:10], disag[0:10])
#print(df.iloc[subset_vector,].text)
#for x,y,z in zip(subset_vector, fp_lab, alc_fp_preds): 
#    print(df.iloc[x].text)
#    print("Label: ", y, " -- prediction: ", z)
#    print("Agreement: ", y == z)
#    print("\n\n")


#for x,y,z in zip(disag, fp_lab, alc_fp_preds): 
#    print(df.iloc[x].text)
#    print("Label: ", y, " -- prediction: ", z)
#    print("Agreement: ", y == z)
#    print("\n\n")
